# Parsing APIs Example

## Intro

Now we will take a look on a real data. When you parse data from web you will often meet API based web-pages. 

For example [zalando.fr](https://www.zalando.fr/accueil-homme/) is API based web-page. 

In this guided lab you will learn how to obtain the links from webpages and extract the data. Read through this doc, execute the cells in order and make sure you understand the explanations. 

*Note: This guided lab uses Google Chrome. Other browsers like Safari and Firefox have similar tools for developers but they work differently. To save your time in following this lab, it is strongly recommended that you install and use Google Chrome.*

## Obtaining the link

Zalando is discount e-store where you can buy clothes and accesories with discount. When we go to the web-page, we can choose different sections. First the general process will be shown using [Children section](https://www.zalando.fr/accueil-enfant/) as example.

Here we will parse data about promotions only. Therefore, final output will be the DataFrame with all the goods under discount.

[![Image from Gyazo](https://i.gyazo.com/fa4874d8e81c7570273bbfb853d66308.png)](https://gyazo.com/fa4874d8e81c7570273bbfb853d66308)


We go to Promos page. Right click of mouse shows us a list of actions possible, from which we select Inspect.

<img src='https://i.gyazo.com/bccbd11d69c9040dc98758d443e32052.png' width="400">


You will see the menu dropdown on the right side or on the bottom of the window. There you should click on Network:


[![Image from Gyazo](https://i.gyazo.com/f7e0db81cbfee67694183d1a7640bf81.png)](https://gyazo.com/f7e0db81cbfee67694183d1a7640bf81)

Right after the developer part will change showing the files behind the page. In order to obtain only useful files we select the following settings:
1. Preserve Log
2. Select XHR files.

[![Image from Gyazo](https://i.gyazo.com/9a899d4441d9d93e795f79747f1e47d5.png)](https://gyazo.com/9a899d4441d9d93e795f79747f1e47d5)

In order to obtain some files we need to scrool down and go forward to second page. 

[![Image from Gyazo](https://i.gyazo.com/0956eb3d5125075a236c9a439c7749c7.png)](https://gyazo.com/0956eb3d5125075a236c9a439c7749c7)

In the Network panel you can see the following files being uploaded. All the data on the web-page is uploaded from the json file, which is one of the following. It is important to understand which file contains what kind of information. 

<a href="https://gyazo.com/cf97a655869f0b22df0ada1cb2a41c3c"><img src="https://i.gyazo.com/cf97a655869f0b22df0ada1cb2a41c3c.png" alt="Image from Gyazo" width="724.8"/></a>

When you find what kind of information you need for the data to be uploaded you just test it. Here we need the article... file:

<a href="https://gyazo.com/78b35bf492994b3f35c0564a21da202a"><img src="https://i.gyazo.com/78b35bf492994b3f35c0564a21da202a.png" alt="Image from Gyazo" width="727.2"/></a>

When we test the link in Chrome inkognito mode we obtain the proper json file:


<a href="https://gyazo.com/b60453fa98454fa29771c731a5174443"><img src="https://i.gyazo.com/b60453fa98454fa29771c731a5174443.png" alt="Image from Gyazo" width="1530.4"/></a>

In order to change the objects in the json file (kind of pagination), you need to change the offset (the number of the first element on the page). in fact, if you take a look on the link, it is easy to unerstand the structure of the link.

# Reading the data

Now the party rocks! When we know how can we obtain the data, it is not a problem to obtain the whole database with all the data from the web-page.
In this lab you will collect your database of Zalando products. You select which goods you want to track. You can define as many filters to your data as you want. Just make sure that the data represents the filters.




In [315]:
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [316]:
# Paste the url you obtained for your data
url='https://www.zalando.fr/api/catalog/articles?categories=promo-enfant&limit=84&offset=84&sort=sale'
headers = {'User-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36'}

#### Collect first 84 object of the of the data (1st page)

Your output should be a Pandas DataFrame of goods. Each row should contain only text or numbers, having *family_articles, flags, media* and *sizes* remaining lists (they are exceptions). Hint: use the headers parameter to get the data!

In [317]:
#response + results
response = requests.get(url, headers =headers)
results = response.json()


In [318]:
#flatten_data_1
flatten_data = json_normalize(results)
flatten_data_1 = json_normalize(flatten_data.articles[0])


#### Collect all the objects from selected filters. Total number of pages can be found in the same json. Use *sku* column as index.

Your output should be a Pandas DataFrame of goods. Each row should contain only text or numbers, having family_articles, flags, media and sizes remaining lists (they are exceptions).

In [319]:
#results

In [320]:
# Get the total number of pages
lst = []
for i in range(269):
    count = 84*i
    urls = 'https://www.zalando.fr/api/catalog/articles?categories=promo-enfant&limit=84&offset='+str(count)+'&sort=sale'
    response_all = requests.get(urls, headers =headers)
    results_all = response_all.json()
    flatten_data_all = json_normalize(results_all)
    flatten_data_all = json_normalize(flatten_data_all.articles[0])
    lst.append(flatten_data_all)
data = pd.concat(lst)
# Your code
data.set_index('sku')

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # This is added back by InteractiveShellApp.init_path()


,amount,brand_name,delivery_promises,family_articles,flags,is_premium,media,name,price.base_price,price.has_different_original_prices,...,price.has_different_promotional_prices,price.has_discount_on_selected_sizes_only,price.original,price.promotional,product_group,sizes,tracking_information.impression_beacon,tracking_information.metrigo_impression_urls,tracking_information.source,url_key
sku,,,,,,,,,,,,,,,,,,,,,
NI116D04L-Q11,NaN,Nike Sportswear,[],"[{'sku': 'NI116D04L-Q11', 'url_key': 'nike-spo...","[{'key': 'discountRate', 'value': 'Jusqu’à -20...",False,[{'path': 'NI/11/6D/04/LQ/11/NI116D04L-Q11@8.j...,COURT BOROUGH MID BOOT WINTERIZED - Chaussures...,NaN,False,...,True,False,"54,95 €","43,95 €",shoe,"[28, 28.5, 29.5, 30, 31.5, 32, 33.5, 34, 35]",https://ccp-et.metrigo.zalan.do/event/sbv?z=e8...,[https://ccp-et.metrigo.zalan.do/event/sbv?z=e...,ccp,nike-sportswear-court-borough-mid-boot-winteri...
NI114D0B9-Q14,NaN,Nike Sportswear,[],"[{'sku': 'NI114D0B9-Q14', 'url_key': 'nike-spo...","[{'key': 'discountRate', 'value': '-20%', 'tra...",False,[{'path': 'NI/11/4D/0B/9Q/14/NI114D0B9-Q14@9.j...,AIR MAX 200 - Baskets basses - black/metallic ...,NaN,False,...,False,False,"104,95 €","83,95 €",shoe,"[35.5, 36.5, 37.5, 38, 38.5, 40]",https://ccp-et.metrigo.zalan.do/event/sbv?z=e8...,[https://ccp-et.metrigo.zalan.do/event/sbv?z=e...,ccp,nike-sportswear-air-max-200-baskets-basses-ni1...
NI114D0C8-Q11,NaN,Nike Sportswear,[],"[{'sku': 'NI114D0C8-Q11', 'url_key': 'nike-spo...","[{'key': 'discountRate', 'value': 'Jusqu’à -20...",False,[{'path': 'NI/11/4D/0C/8Q/11/NI114D0C8-Q11@8.j...,SHOX R4 BG - Baskets basses - enigma,NaN,False,...,True,False,"114,95 €","91,95 €",shoe,"[35.5, 36, 36.5, 37.5, 38, 38.5, 39, 40]",https://ccp-et.metrigo.zalan.do/event/sbv?z=e8...,[https://ccp-et.metrigo.zalan.do/event/sbv?z=e...,ccp,nike-sportswear-shox-r4-bg-baskets-basses-enig...
AD116D007-A11,NaN,adidas Originals,[],"[{'sku': 'AD116D007-A11', 'url_key': 'adidas-o...","[{'key': 'discountRate', 'value': 'Jusqu’à -30...",False,[{'path': 'AD/11/6D/00/7A/11/AD116D007-A11@12....,STAN SMITH - Baskets basses - white,NaN,False,...,True,False,"54,95 €","38,45 €",shoe,"[28, 29, 30, 31, 32, 33, 34, 35, 28 1/2, 30 1/...",NaN,NaN,NaN,adidas-originals-stan-smith-baskets-basses-bla...
AD126L00J-Q11,NaN,adidas Originals,[],"[{'sku': 'AD126L00J-Q11', 'url_key': 'adidas-o...","[{'key': 'discountRate', 'value': 'Jusqu’à -35...",False,[{'path': 'AD/12/6L/00/JQ/11/AD126L00J-Q11@8.j...,JACKET - Veste d'hiver - black/white,NaN,False,...,True,False,"64,95 €","41,95 €",clothing,"[3-4a, 4-5a, 5-6a, 6-7a, 7-8a]",NaN,NaN,NaN,adidas-originals-jacket-veste-dhiver-blackwhit...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NA823F0ZU-K11,NaN,Name it,[],"[{'sku': 'NA823F0ZU-K11', 'url_key': 'name-it-...",[],False,[{'path': 'NA/82/3F/0Z/UK/11/NA823F0ZU-K11@7.j...,NKFOMANA DRESS - Robe d'été - dark sapphire,NaN,False,...,False,False,"29,99 €","29,99 €",clothing,"[8a, 11a]",NaN,NaN,NaN,name-it-nkfomana-dress-robe-dete-dark-sapphire...
LE314G02K-K11,NaN,Superfit,[],"[{'sku': 'LE314G02K-K11', 'url_key': 'superfit...","[{'key': 'discountRate', 'value': 'Jusqu’à -20...",False,[{'path': 'LE/31/4G/02/KK/11/LE314G02K-K11@8.j...,MIKE - Sandales - blau,NaN,True,...,False,False,"54,95 €","54,95 €",shoe,"[29, 33, 34]",NaN,NaN,NaN,superfit-mike-sandales-de-randonnee-blau-le314...
N1243D12J-Q11,NaN,Nike Performance,[],"[{'sku': 'N1243D12J-Q11', 'url_key': 'nike-per...",[],False,[{'path': 'N1/24/3D/12/JQ/11/N1243D12J-Q11@5.j...,INTER MAILAND DRY - Article de supporter - black,NaN,False,...,False,False,"55,00 €","55,00 €",clothing,[8-10a],NaN,NaN,NaN,nike-performance-inter-mailand-dry-t-shirt-a-m...


#### Display the trending brand in DataFrame

In [321]:
# your code
data.columns

Index(['amount', 'brand_name', 'delivery_promises', 'family_articles', 'flags',
       'is_premium', 'media', 'name', 'price.base_price',
       'price.has_different_original_prices', 'price.has_different_prices',
       'price.has_different_promotional_prices',
       'price.has_discount_on_selected_sizes_only', 'price.original',
       'price.promotional', 'product_group', 'sizes', 'sku',
       'tracking_information.impression_beacon',
       'tracking_information.metrigo_impression_urls',
       'tracking_information.source', 'url_key'],
      dtype='object')

In [322]:
#trending brand Name it: 1026
data['brand_name'].value_counts()

Name it            1020
GAP                 713
Esprit              679
Boboli              668
Benetton            526
                   ... 
Umbro                 1
Deuter                1
18CRR81 Cerruti       1
Weleda                1
Cat Footwear          1
Name: brand_name, Length: 349, dtype: int64

#### Display the brand with maximal total discount (sum of discounts on all goods)

In [323]:
#Our data is still text. Convert prices into numbers.

# your code
import re
def string_to_float(df,cl):
    '''
     **import re**
    - extracts numbers with a comma in between from a string
    - list comprehension that iterates through the column and turns the comma into a dot
    - turn the column into a float
    
    input: df[cl] type == str
    output: df[cl] type == float
    
    '''
    df[cl] = df[cl].str.extract('(\d*,\d*)')
    df[cl] = [i.replace(',', '.') for i in df[cl]]
    df[cl] = df[cl].astype(float)
    return df[cl]

In [324]:
string_to_float(data, 'price.original')

0      54.95
1     104.95
2     114.95
3      54.95
4      64.95
       ...  
25     29.99
26     54.95
27     55.00
28     54.95
29     40.00
Name: price.original, Length: 22038, dtype: float64

In [325]:
string_to_float(data, 'price.promotional')

0     43.95
1     83.95
2     91.95
3     38.45
4     41.95
      ...  
25    29.99
26    54.95
27    55.00
28    54.95
29    40.00
Name: price.promotional, Length: 22038, dtype: float64

In [326]:
data['discount'] = data['price.original'] - data['price.promotional']

In [328]:
brand = data.groupby('brand_name', as_index = False)['discount'].sum()
brand

,brand_name,discount
0,18CRR81 Cerruti,25.50
1,3 Pommes,466.05
2,8848 Altitude,347.50
3,ASICS,508.70
4,Abercrombie & Fitch,1118.05
...,...,...
344,pick & PACK,46.70
345,pure pure by BAUER,114.70
346,s.Oliver,2384.30
347,shoeb76,548.00


#### Display the brands without discount at all

In [ ]:
# your code
